# Hands-On Assignment 5

In this assignment, you will practice everything that you have learned so far in an end-to-end setting.
You will be provided with a dataset that is **unique to you**, and your task is to perform
all the steps from previous assignments to clean, explore, visualize, and analyze your dataset.

**Written Portion**: Additionally, you will create a report that describes your process and provides insights about your dataset.
Each section that should appear in your report is noted with an orange star (like normal HO tasks).  The report should be  4-6 pages (12 pt font, 1.5 line spacing), and turned in on Canvas as a PDF.

The coding aspect for this assignment will be turned in the same was as all other HO's,
by submitting this file to the autograder.


For this assignment, feel free to make additional functions instead of implementing everything in the provided function.

The objective of this assignment is for you to apply and solidify the skills you have learned in previous assignments.

# Prompt

You have graduated from this class, and are a huge success!
You landed a job doing data science at some fancy company.

You just got a new client with some really interesting problems you get to solve.
Unfortunately, because of a big mess-up on their side the data's metadata got corrupted up
(and the person that used to maintain the data just took a vow of silence and moved to a bog).

The only column you are sure about is the `label` column,
which contains a numeric label for each row.
Aside from that, the client does not know anything about the names, content, or even data types for each column.

Your task is to explore, clean, and analyze this data.
You should have already received an email with the details on obtaining your unique data.
Place it in the same directory as this notebook (and your `local_grader.py` script) and name it `data.txt`.

*I know this prompt may sound unrealistic, but I have literally been in a situation exactly like this.
I was working at a database startup, and one of our clients gave us data with over 70 columns and more than a million records and told us:
"The person who used to manage the data is no longer working with us, but this was the data they used to make all their decisions.
We also lost all the metadata information, like column names."
...
Working in industry is not always glamorous.
-Eriq*

# Part 0: Explore Your Data

Before you start doing things to/with your data, it's always a good idea to load up your data and take a look.

In [43]:
import pandas
import re
import sklearn.linear_model
import sklearn.ensemble
import sklearn.svm
from sklearn.model_selection import cross_validate
from scipy.stats import ttest_ind

# Modify this to point to your data.
unique_data = pandas.read_csv('data.txt', sep = "\t")
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10,col_11
0,2,812 mph,0.0504 kg/s,sacramento,REPTILE,0.3522 m^2,56,0.2101,0.9054,578 kg/s,dog,0.3391,426 m^3
1,2,817 mph,0.2736 kg/s,Modesto,bird,0.2333 m^2,238,0.4489,1.1045,132 kg/s,dog,0.4947,591 m^3
2,0,397 mph,0.2755 kg/s,San Jose,guinea pig,0.7766 m^2,761,0.8635,0.4515,253 kg/s,reptile,0.9268,-27 m^3
3,1,908 mph,0.219 kg/s,Irvine,rabbit,0.6577 m^2,631,0.8218,0.0901,612 kg/s,guinea pig,0.1285,429 m^3
4,0,9 mph,0.2077 kg/s,San Jose,guinea pig,0.8752 m^2,562,0.7052,0.5017,261 kg/s,reptile,0.766,62 m^3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,1,900 mph,0.2378 kg/s,Riverside,ferret,0.7547 m^2,549,0.7518,0.3346,408 kg/s,guinea pig,0.3005,510 m^3
1480,2,828 mph,0.2821 kg/s,Fremont,bird,0.4151 m^2,100,0.3324,0.8788,240 kg/s,fish,0.6195,578 m^3
1481,1,824 mph,0.4447 kg/s,Riverside,ferret,0.7808 m^2,666,0.5610,0.566,667 kg/s,guinea pig,0.1128,398 m^3
1482,4,824 mph,0.7833 kg/s,Chula Vista,chicken,0.4668 m^2,669,0.3364,0.0887,396 kg/s,ferret,0.5459,367 m^3


Don't forget to checkout the column information.

In [44]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1484 non-null   int64  
 1   col_00  1474 non-null   object 
 2   col_01  1470 non-null   object 
 3   col_02  1471 non-null   object 
 4   col_03  1470 non-null   object 
 5   col_04  1472 non-null   object 
 6   col_05  1475 non-null   object 
 7   col_06  1467 non-null   float64
 8   col_07  1473 non-null   object 
 9   col_08  1472 non-null   object 
 10  col_09  1473 non-null   object 
 11  col_10  1471 non-null   object 
 12  col_11  1470 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 150.8+ KB


And any numeric information.

In [45]:
unique_data.describe()

,label,col_06
count,1484.000000,1467.000000
mean,2.486523,0.491192
std,1.712028,0.317590
min,0.000000,-0.350200
25%,1.000000,0.243850
50%,3.000000,0.525800
75%,4.000000,0.732500
max,5.000000,1.293300


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Introduction</h4>

Briefly describe the dataset you’re given and define the goal of the project and how you approach it.
For example, you can present a basic introduction of your data (shape and proposed data types)
and your goal is to use these features to predict the label of the response variable.
Then you propose a few models that are suitable for this project which will be introduced in the modeling section.

# Part 1: Data Cleaning

As always, we should start with data cleaning.
Take what you learned from HO3 to clean up this messy data to a point where it is ready for machine learning algorithms.

Some things you may want to do:
 - Deal with missing/empty values.
 - Fix numeric columns so that they actually contain numbers.
 - Remove inconsistencies from columns.
 - Assign a data type to each column.

<h4 style="color: darkorange; font-size: x-large";>★ Task 1.A</h4>

Complete the following function that takes in a DataFrame and outputs a clean version of the DataFrame.
You can assume that the frame has all the same structure as your unique dataset.
You can return the same or a new data frame.

In [46]:
def clean_data(frame, ignore_columns=[]):
    cleaned_frame = frame.copy()

    for column in cleaned_frame.columns:
        if column in ignore_columns:
            continue

        cleaned_frame[column] = cleaned_frame[column].apply(lambda x: clean_cell(x))

    return guess_types(cleaned_frame)


def clean_cell(cell_value):
    if not isinstance(cell_value, str):
        return 0

    str_nums = re.findall(r'-?\d+(?:[,.]\d+)*', cell_value)

    if len(str_nums) == 0:
        return 0

    num = str_nums[0].replace(",", "")

    if "." in num:
        return float(num)
    else:
        return int(num)


def guess_types(frame):
    data_types = {}

    for column in frame.columns:
        column_data = frame[column]

        if column_data.dtype == object:
            if column_data.str.match(r'^[-+]?[0-9]*\.?[0-9]+$').all():
                data_types[column] = float
            else:
                data_types[column] = str
        elif column_data.dtype == int:
            data_types[column] = int
        elif column_data.dtype == float:
            data_types[column] = float

    return frame.astype(data_types)


unique_data = clean_data(unique_data)
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10,col_11
0,0,812,0.0504,0,0,0.3522,56,0,0.9054,578,0,0.3391,426
1,0,817,0.2736,0,0,0.2333,238,0,1.1045,132,0,0.4947,591
2,0,397,0.2755,0,0,0.7766,761,0,0.4515,253,0,0.9268,-27
3,0,908,0.2190,0,0,0.6577,631,0,0.0901,612,0,0.1285,429
4,0,9,0.2077,0,0,0.8752,562,0,0.5017,261,0,0.7660,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,0,900,0.2378,0,0,0.7547,549,0,0.3346,408,0,0.3005,510
1480,0,828,0.2821,0,0,0.4151,100,0,0.8788,240,0,0.6195,578
1481,0,824,0.4447,0,0,0.7808,666,0,0.5660,667,0,0.1128,398
1482,0,824,0.7833,0,0,0.4668,669,0,0.0887,396,0,0.5459,367


Now we should also be able to view all the numeric columns.

In [7]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1484 non-null   int64  
 1   col_00  1484 non-null   int64  
 2   col_01  1484 non-null   float64
 3   col_02  1484 non-null   int64  
 4   col_03  1484 non-null   int64  
 5   col_04  1484 non-null   float64
 6   col_05  1484 non-null   int64  
 7   col_06  1484 non-null   int64  
 8   col_07  1484 non-null   float64
 9   col_08  1484 non-null   int64  
 10  col_09  1484 non-null   int64  
 11  col_10  1484 non-null   float64
 12  col_11  1484 non-null   int64  
dtypes: float64(4), int64(9)
memory usage: 150.8 KB


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Cleaning</h4>

Describe the steps you took for data cleaning.
Why did you do this?
Did you have to make some choices along the way? If so, describe them.

# Part 2: Data Visualization

Once you have cleaned up the data, it is time to explore it and find interesting things.
Part of this exploration, will be visualizing the data in a way that makes it easier for yourself and others to understand.
Use what you have learned in HO1 and HO2 to create some visualizations for your dataset.

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Visualization</h4>

Create at least two different visualizations that help describe what you see in your dataset.
Include these visualizations in your report along with descriptions of
how you created the visualization,
what data preparation you had to do for the visualization (aside from the data cleaning in the previous part),
and what the visualization tells us about the data.

# Part 3: Modeling

Now that you have a good grasp of your clean data,
it is time to do some machine learning!
(Technically all our previous steps were also machine learning,
but now we get to use classifiers!)

Use the skills you developed to select **three** classifiers and implement them on your data.
For example, you can narrow down your choices to three classifiers which may include:
- Logistic regression
- K-nearest neighbors
- Decision tree
- Or others

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.A</h4>

Complete the following function that takes in no parameters,
and returns a list with **three** untrained classifiers you are going to explore in this assignment.
This method may set parameters/options for the classifiers, but should not do any training/fitting.

For example, if you wanted to use logistic regression,
then **one** of your list items may be:
```
sklearn.linear_model.LogisticRegression()
```

In [47]:
def create_classifiers():
    classifiers = [
        sklearn.linear_model.LogisticRegression(),
        sklearn.ensemble.RandomForestClassifier(),
        sklearn.svm.SVC()
    ]
    return classifiers

my_classifiers = create_classifiers()
my_classifiers

[LogisticRegression(), RandomForestClassifier(), SVC()]

Now that we have some classifiers, we can see how they perform.

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.B</h4>

Complete the following function that takes in an untrained classifier, a DataFrame, and a number of folds.
This function should run k-fold cross validation with the classifier and the data,
and return a list with the accuracy of each run of cross validation.
You can assume that the frame has the column `label` and the rest of the columns can be considered clean numeric features.

Note that you may have to break your frame into features and labels to do this.
Do not change the passed-in frame (make copies instead).

If you are getting any `ConvergenceWarning`s you may either ignore them,
or try and address them
(they will not affect your autograder score, but may be something to discuss in the written portion of this assignment).

In [49]:
def check_label_distribution(frame):
    print("Unique labels in the dataset:", frame['label'].unique())
    print("Label distribution:\n", frame['label'].value_counts())

# Function to perform cross-validation
def simulate_another_class(frame, n_samples=100):
    new_class_data = frame.sample(n_samples, replace=True).copy()
    new_class_data['label'] = 1  # Assign a new class
    return new_class_data

# Simulate another class
simulated_data = simulate_another_class(unique_data)
augmented_data = pandas.concat([unique_data, simulated_data])

# Check the new label distribution
check_label_distribution(augmented_data)

def cross_fold_validation(classifier, frame, folds):
    labels = frame['label']
    feature_cols = list(frame.columns)
    feature_cols.remove('label')
    features = frame[feature_cols]

    cv_scores = cross_validate(classifier, features, labels, cv=folds)

    return list(cv_scores['test_score'])
    
# Re-run cross-validation with the augmented dataset
my_classifiers_scores = []
for classifier in my_classifiers:
    accuracy_scores = cross_fold_validation(classifier, augmented_data, 5)
    my_classifiers_scores.append(accuracy_scores)
    print("Classifier: %s, Accuracy: %s." % (type(classifier).__name__, accuracy_scores))

Unique labels in the dataset: [0 1]
Label distribution:
 label
0    1484
1     100
Name: count, dtype: int64
Classifier: LogisticRegression, Accuracy: [0.9369085173501577, 0.9369085173501577, 0.9369085173501577, 0.9369085173501577, 0.9367088607594937].


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

Classifier: RandomForestClassifier, Accuracy: [0.8958990536277602, 0.8990536277602523, 0.8832807570977917, 0.8958990536277602, 0.8860759493670886].
Classifier: SVC, Accuracy: [0.9369085173501577, 0.9369085173501577, 0.9369085173501577, 0.9369085173501577, 0.9367088607594937].


<h4 style="color: darkorange; font-size: x-large";>★ Task 3.C</h4>

Complete the following function that takes in two equally-sized lists of numbers and a p-value.
This function should compute whether there is a statistical significance between
these two lists of numbers using a [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test)
at the given p-value.
Return `True` if there is a statistical significance, and `False` otherwise.
Hint: If you wish, you may use the `ttest_ind()` [method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) provided in the scipy package. 

In [20]:
def significance_test(a_values, b_values, p_value):
    _, p = ttest_ind(a_values, b_values)
    return p < p_value

for i in range(len(my_classifiers)):
    for j in range(i + 1, len(my_classifiers)):
        significant = significance_test(my_classifiers_scores[i], my_classifiers_scores[j], 0.10)
        print("%s vs %s: %s" % (type(my_classifiers[i]).__name__,
                                type(my_classifiers[j]).__name__, significant))

LogisticRegression vs RandomForestClassifier: True
LogisticRegression vs SVC: False
RandomForestClassifier vs SVC: True


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Modeling</h4>

Describe the classifiers you have chosen.
Be sure to include all details about any parameter settings used for the algorithms.

Compare the performance of your models using k-fold validation.
You may look at accuracy, F1 or other measures.

Then, briefly summarize your results.
Are your results statistically significant?
Is there a clear winner?
What do the standard deviations look like, and what do they tell us about the different models?
Include a table like Table 1.

<center>Table 1: Every table need a caption.</center>

| Model | Mean Accuracy | Standard Deviation of Accuracy |
|-------|---------------|--------------------------------|
| Logistic Regression | 0.724 | 0.004
| K-Nearest Neighbor | 0.750 | 0.003
| Decision Tree | 0.655 | 0.011

# Part 4: Analysis

Now, take some time to go over your results for each classifier and try to make sense of them.
 - Why do some classifiers work better than others?
 - Would another evaluation metric work better than vanilla accuracy?
 - Is there still a problem in the data that should fixed in data cleaning?
 - Does the statistical significance between the different classifiers make sense?
 - Are there parameters for the classifier that I can tweak to get better performance?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Analysis</h4>

Discuss your observations, the relationship you found, and how you applied concepts from the class to this project.
For example, you may find that some feature has the most impact in predicting your response variable or removing a feature improves the model accuracy.
Or you may observe that your training accuracy is much higher than your test accuracy and you may want to explain what issues may arise.

# Part 5: Conclusion

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Conclusion</h4>

Briefly summarize the important results and conclusions presented in the project.
What are the important points illustrated by your work?
Are there any areas for further investigation or improvement?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: References</h4>

Include a standard bibliography with citations referring to techniques or published papers you used throughout your report (if you used any).

For example:
```
[1] Derpanopoulos, G. (n.d.). Bayesian Model Checking & Comparison.
https://georgederpa.github.io/teaching/modelChecking.html.
```

# Part XC: Extra Credit

So far you have used a synthetic dataset that was created just for you.
But, data science is always more interesting when you are dealing with actual data from the real world.
Therefore, you will have an opportunity for extra credit on this assignment using real-world data.

For extra credit, repeat the **written tasks** of Parts 0 through 4 with an additional dataset that you find yourself.
For the written portion of the extra credit for Part 0, include information about where you got the data and what the data represents.
You may choose any dataset that represents real data (i.e., is **not** synthetic or generated)
and is **not** [pre-packaged in scikit-learn](https://scikit-learn.org/stable/datasets.html).

Below are some of the many places you can start looking for datasets:
 - [Kaggle](https://www.kaggle.com/datasets) -- Kaggle is a website focused around machine learning competitions,
       where people compete to see who can get the best results on a dataset.
       It is very popular in the machine learning community and has thousands of datasets with descriptions.
       Make sure to read the dataset's description, as Kaggle also has synthetic datasets.
 - [data.gov](https://data.gov/) -- A portal for data from the US government.
        The US government has a lot of data, and much of it has to be available to the public by law.
        This portal contains some of the more organized data from several different government agencies.
        In general, the government has A LOT of interesting data.
        It may not always be clean (remember the CIA factbook), but it is interesting and available.
        All data here should be real-world, but make sure to read the description to verify.
 - [UCI's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) -- UC Irvine has their own data repository with a few hundred datasets on many different topics.
        Make sure to read the dataset's description, as UCI also has synthetic datasets.
 - [WHO's Global Health Observatory](https://apps.who.int/gho/data/node.home) -- The World Health Organization keeps track of many different health-related statistics for most of the countries in the world.
        All data here should be real-world, but make sure to read the description to verify.
 - [Google's Dataset Search](https://datasetsearch.research.google.com/) -- Google indexes many datasets that can be searched here.

You can even create a dataset from scratch if you find some data you like that is not already organized into a specific dataset.
The only real distinction between "data" and a "dataset" is that a dataset is organized and finite (has a fixed size).

Create a new section in your written report for this extra credit and include all the written tasks for the extra credit there.
Each written task/section that you complete for your new dataset is eligible for extra credit (so you can still receive some extra credit even if you do not complete all parts).
There is no need to submit any code for the extra credit.
If you created a new dataset, include the dataset or links to it with your submission.